<a href="https://colab.research.google.com/github/wllgrnt/keras-examples/blob/master/Chapter5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 5

## Deep Learning for Computer Vision

In [1]:
import keras

Using TensorFlow backend.


### Convolutional networks

Here's a basic MNIST classifier made up `Conv2D` and `MaxPooling2D` layers

In [2]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, (3,3), activation="relu", input_shape=(28,28,1)))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Conv2D(64, (3,3), activation="relu"))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Conv2D(64, (3,3), activation="relu"))

W0614 19:31:39.983044 140068333918080 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0614 19:31:40.007148 140068333918080 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0614 19:31:40.010866 140068333918080 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0614 19:31:40.047506 140068333918080 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



In [3]:
# Display the model architecture
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


In [4]:
# Connect this convnet to a classifier network - a stack of `Dense` layers
# These process vectors, so we have to flatten the 3D tensor

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation="relu"))
model.add(keras.layers.Dense(10, activation="softmax"))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
__________

In [5]:
# Now train on MNIST
(train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()


train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype("float32") /255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype("float32") /255

train_labels = keras.utils.to_categorical(train_labels)
test_labels = keras.utils.to_categorical(test_labels)

model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

11493376/11490434 [==============================] - 0s 0us/step


W0614 19:31:43.742312 140068333918080 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0614 19:31:43.770268 140068333918080 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0614 19:31:43.882131 140068333918080 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0614 19:31:44.002884 140068333918080 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated

Epoch 1/5
60000/60000 [==============================] - 53s 879us/step - loss: 0.1782 - acc: 0.9445
Epoch 2/5
60000/60000 [==============================] - 53s 879us/step - loss: 0.0473 - acc: 0.9854
Epoch 3/5
60000/60000 [==============================] - 53s 886us/step - loss: 0.0336 - acc: 0.9901
Epoch 4/5
60000/60000 [==============================] - 53s 885us/step - loss: 0.0259 - acc: 0.9923
Epoch 5/5
60000/60000 [==============================] - 53s 876us/step - loss: 0.0213 - acc: 0.9934


In [6]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_acc:.4f}")

10000/10000 [==============================] - 3s 301us/step
Test accuracy: 0.9919


### ConvNets on the Cats and Dogs dataset

Here we'll look at a sample of 4,000 photos of cats and dogs, and train on 2,000 of them. Naively training a convnet will result in overfitting, which we'll sort with data augmentation. Then we'll use a pre-trained model to get top accuracy.

In [23]:
from google.colab import drive, files
import os
import shutil
import zipfile

drive.mount('/content/drive/')
path = "/content/drive/My Drive/Colab Notebooks/dogs-vs-cats.zip"

zip_ref = zipfile.ZipFile(path, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

# For odd historic reasons, the directory is called train
shutil.move("/tmp/train", "/tmp/dogs_vs_cats")


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


'/tmp/dogs_vs_cats'

In [24]:
!ls /tmp

cats_and_dogs.zip  drivefs_ipc.0	sampleSubmission.csv  train.zip
dogs_vs_cats	   drivefs_ipc.0_shell	test1.zip


In [25]:
# Create the test_train directory structure

original_dir = '/tmp/dogs_vs_cats'

base_dir = "/tmp/cats_and_dogs_small"
os.mkdir(base_dir)
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)


# Directory with our training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')
os.mkdir(train_cats_dir)
# Directory with our training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')
os.mkdir(train_dogs_dir)

# As above, for the validation set
validation_cats_dir = os.path.join(validation_dir, 'cats')
os.mkdir(validation_cats_dir)
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
os.mkdir(validation_dogs_dir)


# As above, for the test set
test_cats_dir = os.path.join(test_dir, 'cats')
os.mkdir(test_cats_dir)
test_dogs_dir = os.path.join(test_dir, 'dogs')
os.mkdir(test_dogs_dir)

# The first 1000 cats are for training
fnames = [f"cat.{i}.jpg" for i in range(1000)]
for fname in fnames:
  src = os.path.join(original_dir, fname)
  dst = os.path.join(train_cats_dir, fname)
  shutil.copyfile(src,dst)
  
# The next 500 are for validation
fnames = [f"cat.{i}.jpg" for i in range(1000,1500)]
for fname in fnames:
  src = os.path.join(original_dir, fname)
  dst = os.path.join(validation_cats_dir, fname)
  shutil.copyfile(src,dst)


# The next 500 are for testing
fnames = [f"cat.{i}.jpg" for i in range(1500,2000)]
for fname in fnames:
  src = os.path.join(original_dir, fname)
  dst = os.path.join(test_cats_dir, fname)
  shutil.copyfile(src,dst)

  
# The first 1000 dogs are for training
fnames = [f"dog.{i}.jpg" for i in range(1000)]
for fname in fnames:
  src = os.path.join(original_dir, fname)
  dst = os.path.join(train_dogs_dir, fname)
  shutil.copyfile(src,dst)
  
# The next 500 are for validation
fnames = [f"dog.{i}.jpg" for i in range(1000,1500)]
for fname in fnames:
  src = os.path.join(original_dir, fname)
  dst = os.path.join(validation_dogs_dir, fname)
  shutil.copyfile(src,dst)


# The next 500 are for testing
fnames = [f"dog.{i}.jpg" for i in range(1500,2000)]
for fname in fnames:
  src = os.path.join(original_dir, fname)
  dst = os.path.join(test_dogs_dir, fname)
  shutil.copyfile(src,dst)

  
print('total training cat images:', len(os.listdir(train_cats_dir)))
print('total training dog images:', len(os.listdir(train_dogs_dir)))
print('total validation cat images:', len(os.listdir(validation_cats_dir)))
print('total validation dog images:', len(os.listdir(validation_dogs_dir)))
print('total test cat images:', len(os.listdir(test_cats_dir)))
print('total test dog images:', len(os.listdir(test_dogs_dir)))

total training cat images: 1000
total training dog images: 1000
total validation cat images: 500
total validation dog images: 500
total test cat images: 500
total test dog images: 500
